# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3 
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()
import pandas as pd

In [ ]:
#Your code here; create the connection and cursor

## Select the Average Number of Orders on a Product by Product Basis

In [ ]:
c.execute("""select AVG(orderNumber) productName
             from orders
             join orderdetails using(orderNumber)
             join products using(productCode)
             group

In [4]:
#Your code here; use a subquery. No join will be necessary.
 c.execute("""select AVG(numOrders)
                    from (select productCode,
                                 count(distinct orderNumber) as numOrders
                                 from products
                                 join orderdetails
                                 using(productCode)
                                 group by 1
                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

109


,productCode,avg(quantityordered)
0,S10_1678,37.750000
1,S10_1949,34.321429
2,S10_2016,35.678571
3,S10_4698,35.178571
4,S10_4757,36.785714


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [5]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select customerNumber, contactLastName, ContactFirstName
                    from customers
                    where customerNumber in (select customerNumber from orders where orderDate = "2003-01-31");
          """
         )
df = pd.DataFrame(c.fetchall())
df. columns = [i[0] for i in c.description]
print(len(df))
df.head()

1


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [6]:
c.execute("""select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';""")
df = pd.DataFrame(c.fetchall())
df. columns = [i[0] for i in c.description]
print(len(df))
df.head()        

1


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [9]:
#Your code here

c.execute("""select productName,
                      count(quantityordered) as Total_Number_Orders
                      from orderdetails
                      join products
                      using(productCode)
                      group by 1
                      order by count(quantityordered);""")
 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df.head()
                      

,productName,Total_Number_Orders
0,1952 Citroen-15CV,24
1,1957 Ford Thunderbird,24
2,1911 Ford Town Car,25
3,1917 Grand Touring Sedan,25
4,1932 Alfa Romeo 8C2300 Spider Sport,25


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [17]:
#Your code here
#c.execute("""select productName, count(distinct customerNumber),
#                      from products
#                      join orderdetails
#                      using(productCode)
#                      join orders
#                      using(orderNumber)
#                      join customers
#                      using(customernumber)
#                      group by 1
#                      order by count(distinct customerNumber);""")
 
c.execute("""select productName,
                    count(distinct customerNumber) numPurchasers
                    from products
                    join orderdetails
                    using(productCode)
                    join orders
                    using (orderNumber)
                    group by 1
                    order by numPurchasers desc
                    ;
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df.head()


,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [18]:
#Your code here
c.execute("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         );
           """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur
5,1286,3,NYC,Foon Yue,Tseng
6,1166,1,San Francisco,Leslie,Thompson
7,1165,1,San Francisco,Leslie,Jennings
8,1612,6,Sydney,Peter,Marsh
9,1188,2,Boston,Julie,Firrelli


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here
c.execute("""select employeeNumber,
                    firstName,
                    lastName,
                    count(distinct customerNumber)
                    from employees e
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    group by 1,2,3
                    having AVG(creditLimit) > 15000;
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!